# 분류 모델을 활용하여 주가 분석

## 가설
- feature를 뉴스 기사 원문으로 하여 target인 등락을 분류 모델로 예측할 수 있지 않을까?

## 방법
1. 데이터 수집
    - 날짜, 뉴스, 종가, 전날 종가, 등락으로 칼럼 구성
    - 삼성전자라는 키워드를 입력하여 그에 대한 기사의 작성 날짜와 뉴스 원문을 크롤링한다.
    - yfinance를 통해 종가 데이터도 수집한다.
    - 등락 데이터는 '종가 - 전날 종가'로 구한다.
---
2. 데이터 전처리
    - 단어 토큰화, 클렌징 등은 라이브러리를 활용하여 수행한다.
        - konlpy
    - BOW 방식 중 TF-IDF 벡터화를 통해 단어들을 벡터화 시킨다.
---
3. 분류 모델 적용
    - 로지스틱 회귀를 활용할 예정이며 다른 분류 모델들도 적용해 정확도를 비교한다,
---
4. 최종 평가
    - 최종 평가를 하며 마무리한다.

## 1. 데이터 수집

In [53]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import pandas as pd

name = "삼성전자"
url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query="+name

response = requests.get(url)

html = response.text

soup = BeautifulSoup(html,'html.parser')
news_url = soup.select_one('#sp_nws1 > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)')
print(news_url['href'])

url2 = news_url['href']

response2 = requests.get(url2)
html2 = response2.text

soup2 = BeautifulSoup(html2,'html.parser')
date = soup2.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')
content = soup2.select_one('#dic_area')
print(date['data-date-time'][0:10])
print(content)

https://n.news.naver.com/mnews/article/001/0014091355?sid=101
2023-07-26
<div class="go_trans _article_content" id="dic_area">
<strong style="display:block;overflow:hidden;position:relative;margin:33px 20px 10px 3px;padding-left:11px;font-weight:bold;border-left: 2px solid #141414;">CEO스코어, 500대 기업 국민연금 가입자 기준 고용현황 조사<br/>올해 상반기 고용 9천314명 늘어…조선·기계·설비 업종 ↑</strong><br/><br/><span class="end_photo_org"><div class="nbd_im_w _LAZY_LOADING_WRAP">
<div class="nbd_a _LAZY_LOADING_ERROR_HIDE" id="img_a1">
<img class="_LAZY_LOADING" data-src="https://imgnews.pstatic.net/image/001/2023/07/26/PCM20190916000004990_P4_20230726062504476.jpg?type=w647" id="img1">
</img></div>
</div><em class="img_desc">상반기 국내 대기업 순고용 증가(CG)<br/>[연합뉴스TV 제공]</em></span><br/><br/>    (서울=연합뉴스) 장하나 기자 = 극심한 경기 침체에도 국내 대기업들이 올해 상반기에 1만명 가까이 고용을 늘린 것으로 집계됐다.<br/><br/>    특히 삼성전자만 유일하게 순고용을 2천명 이상 늘려 500대 기업 중 순고용 증가 폭이 가장 컸다.<br/><br/>26일 기업데이터연구소 CEO스코어에 따르면 매출액 기준 500대 기업 중 합병·분할 등이 있었던 25곳을 제외한 475곳의 국민연금 가입자 기준 고용 현황을 

In [ ]:
urls = []

pages = 10

for p in range(pages):
    url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query="+name

    response = requests.get(url)

    html = response.text

    soup = BeautifulSoup(html,'html.parser')
    news_url = soup.select_one('#sp_nws1 > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)')
    print(news_url['href'])

In [38]:
import yfinance as yf

start = "2020-02-01"
end = "2023-03-01"
symbol = "005930.KS"

samsung = yf.download(symbol,start)

samsung.tail(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-13,72400.0,72600.0,71900.0,71900.0,71900.0,14417279
2023-07-14,72500.0,73400.0,72400.0,73400.0,73400.0,15882519
2023-07-17,73200.0,73500.0,72800.0,73300.0,73300.0,10060049
2023-07-18,73200.0,73500.0,72000.0,72000.0,72000.0,11697900
2023-07-19,72700.0,72800.0,71300.0,71700.0,71700.0,10896412
2023-07-20,71100.0,71500.0,70800.0,71000.0,71000.0,9732730
2023-07-21,70400.0,70400.0,69400.0,70300.0,70300.0,16528926
2023-07-24,70100.0,70900.0,69900.0,70400.0,70400.0,13418597
2023-07-25,70000.0,70500.0,69800.0,70000.0,70000.0,14314945


In [41]:
samsung.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 861 entries, 2020-02-03 to 2023-07-26
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       861 non-null    float64
 1   High       861 non-null    float64
 2   Low        861 non-null    float64
 3   Close      861 non-null    float64
 4   Adj Close  861 non-null    float64
 5   Volume     861 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 47.1 KB


In [43]:
samsung.index

DatetimeIndex(['2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
               '2020-02-07', '2020-02-10', '2020-02-11', '2020-02-12',
               '2020-02-13', '2020-02-14',
               ...
               '2023-07-13', '2023-07-14', '2023-07-17', '2023-07-18',
               '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-24',
               '2023-07-25', '2023-07-26'],
              dtype='datetime64[ns]', name='Date', length=861, freq=None)

In [47]:
samsung['Close']['2020-02-28']

54200.0

In [52]:
samsung['Close'][-1]-samsung['Close'][-2]

200.0